<h2>Some test with GeoPy and the data from Flex Matcher</h2>

In [16]:
import pandas as pd

df = pd.read_csv(r"..\flex_matcher\merged_companies.csv")

df["city"] = df["city"].str.replace(r"[.,()]", "", regex=True)  

df["full_address"] = df["address"].fillna('') + " " + df["city"].fillna('') + " " + df["country"].fillna('')

df["full_address"] = df["full_address"].str.replace(r"\s+", " ", regex=True) 
df["full_address"] = df["full_address"].str.strip()

print(df[["address", "city", "country", "full_address"]].head())

df_subset = df.head(200)



C:\Users\matte\AppData\Local\Temp\ipykernel_5700\2893365244.py:3: DtypeWarning: Columns (0,12,13,15,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"..\flex_matcher\merged_companies.csv")


  address                                  city country  \
0     NaN              Mumbai Maharashtra India     NaN   
1     NaN                        Dublin Ireland     NaN   
2     NaN  Teaneck New Jersey United States USA     NaN   
3     NaN              Mumbai Maharashtra India     NaN   
4     NaN              Mumbai Maharashtra India     NaN   

                           full_address  
0              Mumbai Maharashtra India  
1                        Dublin Ireland  
2  Teaneck New Jersey United States USA  
3              Mumbai Maharashtra India  
4              Mumbai Maharashtra India  


In [18]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

geolocator = Nominatim(user_agent="my_geocoder")

def get_coordinates(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return pd.Series([location.latitude, location.longitude])
    except GeocoderTimedOut:
        time.sleep(1)
        return get_coordinates(address)
    return pd.Series([None, None])

df_subset[["latitude", "longitude"]] = df_subset["full_address"].apply(get_coordinates)

df_subset.to_csv("subset_with_coordinates.csv", index=False)

print(df_subset)
print(df_subset.shape)


    address                    category                                  city  \
0       NaN    IT Services & Consulting              Mumbai Maharashtra India   
1       NaN    IT Services & Consulting                        Dublin Ireland   
2       NaN    IT Services & Consulting  Teaneck New Jersey United States USA   
3       NaN                     Banking              Mumbai Maharashtra India   
4       NaN                     Banking              Mumbai Maharashtra India   
..      ...                         ...                                   ...   
195     NaN  Engineering & Construction             Kolkata West Bengal India   
196     NaN                      Pharma              Mumbai Maharashtra India   
197     NaN                  Automobile        Gurgaon/Gurugram Haryana India   
198     NaN    IT Services & Consulting   VilniusVilniaus Apskritis Lithuania   
199     NaN                   Logistics   Bangalore/Bengaluru Karnataka India   

    country employees found

C:\Users\matte\AppData\Local\Temp\ipykernel_5700\2834182431.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[["latitude", "longitude"]] = df_subset["full_address"].apply(get_coordinates)


In [ ]:
df = pd.read_csv('subset_with_coordinates.csv')

df_linked = df.groupby(['latitude', 'longitude'], as_index=False).agg({
    'address': 'first',
    'category': 'first',
    'city': 'first',
    'country': 'first',
    'employees': 'first',
    'founded': 'first',
    'fundation_year': 'first',
    'investors': 'first',
    'link': 'first',
    'market_cap': 'first',
    'name': 'first',
    'ownership': 'first',
    'revenue': 'first',
    'satus': 'first',
    'stage': 'first',
    'telephone': 'first',
    'total_raised': 'first',
    'type': 'first',
    'unnamed': 'first',
    'valuation': 'first',
    'full_address': 'first',
})
#sicuramente questo approccio di first non è il migliore, ma testing vabene
#questa operazione di match viene affrontata bene nel record linkage

df_linked.to_csv('linked_subset.csv', index=False)

print(df_linked.head())

#Ordina il csv generato nel precedente step per vedere se ci ha preso!


    latitude  longitude  address                  category  \
0   8.488227  76.947551      NaN                      NBFC   
1   9.871099  76.278277      NaN                      NBFC   
2  10.516140  76.208462      NaN                      NBFC   
3  12.988157  77.622600      NaN  IT Services & Consulting   
4  13.083694  80.270186      NaN                Automobile   

                                         city  country  employees  founded  \
0  Trivandrum/Thiruvananthapuram Kerala India      NaN        NaN      NaN   
1                   Kochi/Cochin Kerala India      NaN        NaN      NaN   
2               Thrissur/Trichur Kerala India      NaN        NaN      NaN   
3         Bangalore/Bengaluru Karnataka India      NaN        NaN      NaN   
4                    Chennai Tamil Nadu India      NaN        NaN      NaN   

       fundation_year  investors  ...  ownership  revenue satus stage  \
0  1887 (136 yrs old)        NaN  ...    Private      NaN   NaN   NaN   
1   1939 (84